# skilleddialog

> Solveit dialog integration for Agent Skills

In [ ]:
#| default_exp skilleddialog

# SkilledDialog

> Extend solveit dialogs with Agent Skills for enhanced capabilities

## Overview

`SkilledDialog` integrates the Agent Skills system with solveit's dialog-based workflow. While `SkilledAgent` wraps claudette agents, `SkilledDialog` works with solveit's message-based architecture.

## Key Differences from SkilledAgent

| Aspect | SkilledAgent (claudette) | SkilledDialog (solveit) |
|--------|--------------------------|-------------------------|
| Tool exposure | `agent.tools.append(t)` | `& \`[tools]\`` backtick notation |
| Instructions | `agent.h.append(msg)` | `add_msg()` from dialoghelper |
| System prompt | `agent.sp` | First message or context |
| Conversation | `Chat` history | Dialog messages |

## Solveit Tool System

Solveit exposes tools to Claude using backtick notation:
- Single tool: `& \`function_name\``
- Multiple tools: `& \`[func1, func2, func3]\``

Functions must have:
- Type annotations for parameters
- Docstrings describing their purpose
- Be importable in the dialog's namespace

## Integration Approach

1. **Discovery** - Reuse `discover_all()` from core
2. **Activation** - Inject instructions via `add_msg()`
3. **Tool exposure** - Import skill tools into namespace, generate backtick declaration
4. **Security** - Reuse `SecurityPolicy` and `validate_capabilities()`

In [ ]:
#| export
from skillhelper.core import *
from skillhelper.core import SecurityPolicy, validate_capabilities

In [ ]:
#| export
from fastcore.basics import AttrDict, patch, first
from pathlib import Path
from dataclasses import dataclass
from typing import Optional, Callable
import sys

In [ ]:
#| export
# Optional imports - only available in solveit environment
try:
    from dialoghelper import add_msg, curr_dialog, mk_toollist, is_usable_tool
    IN_SOLVEIT = True
except ImportError:
    IN_SOLVEIT = False

## SkilledDialog Class

The main class for integrating skills with solveit dialogs.

In [ ]:
#| export
class SkilledDialog:
    "Extend solveit dialog with Agent Skills"
    def __init__(self, policy:SecurityPolicy=None):
        self.policy = policy
        self.active_skills = {}
        self._tool_namespace = {}  # Holds imported tool functions

In [ ]:
sd = SkilledDialog()
sd.policy, sd.active_skills

## Skill Discovery

Reuse the discovery mechanism from core.

In [ ]:
#| export
@patch
def list_skills(self:SkilledDialog):
    "List all available skills"
    return discover_all()

In [ ]:
sd.list_skills()

## Skills Summary for Context

Generate a summary of available skills that can be injected into dialog context.

In [ ]:
#| export
@patch
def _skills_summary(self:SkilledDialog) -> str:
    "Return formatted skills metadata for dialog context"
    skills = discover_all()
    if not skills: return ""
    lines = ["## Available Skills", ""]
    for s in skills:
        lines.append(f"- **{s.name}**: {s.description}")
    lines.append("")
    lines.append("To activate a skill, use `sd.activate_skill('skill-name')`")
    return "\n".join(lines)

In [ ]:
print(sd._skills_summary())

## Tool Declaration Generation

Generate the solveit backtick notation for exposing skill tools.

In [ ]:
#| export
@patch
def tool_declaration(self:SkilledDialog, skill_name:str=None) -> str:
    """Generate solveit backtick declaration for skill tools.
    
    If skill_name is provided, returns declaration for that skill.
    If None, returns declaration for all active skill tools.
    """
    if skill_name:
        skill = self.active_skills.get(skill_name)
        if not skill: return ""
        tools = list(skill.get_tools().keys())
    else:
        tools = list(self._tool_namespace.keys())
    
    if not tools: return ""
    if len(tools) == 1:
        return f"& `{tools[0]}`"
    return f"& `[{', '.join(tools)}]`"

## Skill Activation

Activate a skill by:
1. Loading and validating against security policy
2. Importing tools into namespace
3. Injecting instructions into dialog (if in solveit)
4. Returning tool declaration for user to include

In [ ]:
#| export
@patch
def activate_skill(self:SkilledDialog, name:str, inject_msg:bool=True) -> AttrDict:
    """Activate a skill for use in solveit dialog.
    
    Args:
        name: Name of the skill to activate
        inject_msg: If True and in solveit, inject instructions via add_msg()
    
    Returns:
        AttrDict with success, skill, tools, declaration, and error fields
    """
    # Find the skill
    skill = first(s for s in discover_all() if s.name == name)
    if not skill:
        return AttrDict(success=False, skill=None, tools=[], declaration="", 
                       error=f"Skill '{name}' not found")
    
    # Security validation
    if self.policy:
        try:
            validate_capabilities(skill, self.policy)
        except ValueError as e:
            return AttrDict(success=False, skill=None, tools=[], declaration="",
                           error=str(e))
    
    # Store active skill
    self.active_skills[name] = skill
    
    # Import tools into namespace
    skill_tools = skill.get_tools()
    for tool_name, tool_func in skill_tools.items():
        self._tool_namespace[tool_name] = tool_func
    
    # Generate tool declaration
    declaration = self.tool_declaration(name)
    
    # Inject instructions into dialog if in solveit
    if inject_msg and IN_SOLVEIT:
        msg_content = f"[Skill activated: {name}]\n\n{skill.instructions}\n\n"
        msg_content += f"**Tools available:** {declaration}"
        add_msg(msg_content, msg_type='note')
    
    return AttrDict(
        success=True,
        skill=skill,
        tools=list(skill_tools.values()),
        declaration=declaration,
        error=None
    )

In [ ]:
sd = SkilledDialog()
result = sd.activate_skill('code-reviewer', inject_msg=False)
result.success, result.declaration, result.error

In [ ]:
# Tools are now in namespace
sd._tool_namespace

## Getting Tools for Global Namespace

In solveit, tools need to be accessible in the global namespace for the backtick notation to work. This method returns the tools dict for easy unpacking.

In [ ]:
#| export
@patch
def get_tools(self:SkilledDialog) -> dict:
    """Get all active skill tools for namespace injection.
    
    Usage in solveit:
        globals().update(sd.get_tools())
    """
    return self._tool_namespace.copy()

In [ ]:
sd.get_tools()

## Deactivating Skills

In [ ]:
#| export
@patch
def deactivate_skill(self:SkilledDialog, name:str) -> bool:
    """Deactivate a skill and remove its tools from namespace."""
    if name not in self.active_skills:
        return False
    
    skill = self.active_skills.pop(name)
    # Remove tools from namespace
    for tool_name in skill.get_tools().keys():
        self._tool_namespace.pop(tool_name, None)
    
    return True

In [ ]:
sd.deactivate_skill('code-reviewer')
sd._tool_namespace

## Convenience Function

A simpler API for quick skill activation in solveit.

In [ ]:
#| export
def use_skill(name:str, policy:SecurityPolicy=None, inject_msg:bool=True) -> AttrDict:
    """Quick skill activation for solveit.
    
    Creates a SkilledDialog, activates the skill, and returns result.
    
    Usage in solveit:
        result = use_skill('code-reviewer')
        globals().update(result.tools_dict)
        # Then include in prompt: & `[check_style, count_lines]`
    
    Args:
        name: Skill name to activate
        policy: Optional security policy
        inject_msg: If True, inject instructions into dialog
    
    Returns:
        AttrDict with success, skill, tools, tools_dict, declaration, error
    """
    sd = SkilledDialog(policy)
    result = sd.activate_skill(name, inject_msg=inject_msg)
    result.tools_dict = sd.get_tools()
    result.skilled_dialog = sd
    return result

In [ ]:
result = use_skill('code-reviewer', inject_msg=False)
print(f"Success: {result.success}")
print(f"Declaration: {result.declaration}")
print(f"Tools: {list(result.tools_dict.keys())}")

## Usage in Solveit

Here's how to use skills in a solveit dialog:

```python
# In a code cell:
from skillhelper.skilleddialog import use_skill

result = use_skill('code-reviewer')
globals().update(result.tools_dict)
print(result.declaration)  # Shows: & `[check_style, count_lines]`
```

```markdown
# In a prompt cell:
Please review this code for style issues:

```python
def foo( x ):return x+1
```

& `[check_style, count_lines]`
```

The skill's instructions are automatically injected as a note message, providing Claude with the procedural knowledge for the skill.

## Show Available Skills

Helper to display available skills in a formatted way.

In [ ]:
#| export  
def show_skills():
    """Display available skills in formatted output."""
    skills = discover_all()
    if not skills:
        print("No skills found.")
        return
    
    print("Available Skills:\n")
    for s in skills:
        print(f"  {s.name}")
        print(f"    {s.description}")
        tools = list(s.get_tools().keys())
        if tools:
            print(f"    Tools: {', '.join(tools)}")
        print()

In [ ]:
show_skills()

## Export

In [ ]:
#| hide
import nbdev; nbdev.nbdev_export()